# Using Docker for backend services with DashT

xxx

## Docker Desktop for Windows

## Kitematic

Visual Docker Container Management on Mac & Windows https://kitematic.com (on GitHub https://github.com/docker/kitematic).

View with Kitematic of the three containers, providing three services:

![2020-03-28_kitematic_3_services.png](2020-03-28_kitematic_3_services.png) [(zoom)](img/2020-03-28_kitematic_3_services.png)

View with Kitematic of the three containers, providing three useful services:

* **nginx** - this is the HTTP / proxy server, a swiss knife providing both files to OpenCPN's _DashT_ but also connecting it to other network based services.
* **InfluxDB v2** - time series database which both collects data from DashT but which is also available to feed it back to _DashT_ but also to other useful services:
* **Grafana** - A monitoring solution which allows you to create more complex dashboard's that would be possible with _DashT_ or with _InfluxDB v2_.

>It is noteworthy that we do not run a Signal K node server locally here in Docker - it may require some physical connection like USB and it is better done with _node.js_ which is also network performance wise a better solution than running it in a Docker instance. For testing and learning it is, of course possible to run Signal K as a Docker instance as well.

## nginx

[nginx](http://nginx.org/en/) _engine x_ is an HTTP and reverse proxy server. It is a true Swiss knife, allowing us to provide and share all services local, like http://localhost:8088 - we need more services, we just add port numbers. With _nginx_'s rirch features and high its reverse proxy we can

* Make _DashT_ network based files available as network service from the local file system, the same in which OpenCPN is installed - no copying is needed
* We can make the code in those files to access _InfluxDB v2_ in parallel with your browser and with _Grafana_ by enabling [CORS](https://en.wikipedia.org/wiki/Cross-origin_resource_sharing) (_Cross-Origin-Resource-Sharing_) so that browser security features gets satisfied

### Define your local paths to make available

In this case we use Docker _bind_ method to map directly back to our file system. This is different than _mount_ of a _volume_ which we are going to use with _Grafana_.

#### HTML and JavaScript files

_DashT_ WebView and JavaScript based instruments needs to be loaded from a HTTP server. In the OpenCPN installation on a Windows system the are typically in `C:\'Program Files (x86)'\OpenCPN\plugins\dashboard_tactics_pi\data\instrujs`

We want to bind that folder to _nginx_ Docker instance's folder `/data/www`

### nginx.conf

It is nice to be edit directly the _nginx_'s configuration file without sometimes complicated tricks. That's why we bind it to a local file system file as well.

We want _nginx_ to configure two local port, to act as proxy server and to deal with CORS so that the localhost served JavaScript files can access services such as the _InfluxDB v2_:

* port 8088 - is mapped in Docker container as port 80, bind back to the host's local file system as explained above
* port 8089 - is a proxy for files served from 8088 wanting to access _influxdb_ server - [CORS Access Control headers](https://en.wikipedia.org/wiki/Cross-origin_resource_sharing#How_CORS_works) are replied to browsers confirming them that this is OK - tested Chrome 80, Firefox 74 and even IE11 (because WebView of wxWidgets is using it).

Create in your home folder a folder named `nginx` and create in it a text file `nginx.conf` with the following contents:

```

user  nginx;
worker_processes  1;

error_log  /var/log/nginx/error.log warn;
pid        /var/run/nginx.pid;


events {
    worker_connections  1024;
}


http {
    include       /etc/nginx/mime.types;
    default_type  application/octet-stream;

    log_format  main  '$remote_addr - $remote_user [$time_local] "$request" '
                      '$status $body_bytes_sent "$http_referer" '
                      '"$http_user_agent" "$http_x_forwarded_for"';

    access_log  /var/log/nginx/access.log  main;

    sendfile        on;
    #tcp_nopush     on;

    keepalive_timeout  65;

    #gzip  on;
    
    server {
      listen 80;
      location / {
        root /data/www;
        autoindex on;
      }
    }
    server {
      listen 8089;
      location / {
        if ($request_method = 'OPTIONS') {
            add_header 'Access-Control-Allow-Origin' 'http://localhost:8088';
            add_header 'Access-Control-Allow-Credentials' 'true' always;
            add_header 'Access-Control-Allow-Methods' 'GET, POST, OPTIONS';
            add_header 'Access-Control-Allow-Headers' 'Authorization,Accept,Origin,DNT,User-Agent,X-Requested-With,If-Modified-Since,Cache-Control,Content-Type,Range';
            add_header 'Access-Control-Max-Age' 1728000;
            add_header 'Content-Type' 'text/plain; charset=utf-8';
            add_header 'Content-Length' 0;
            return 204;
        }
        if ($request_method = 'POST') {
            add_header 'Access-Control-Allow-Credentials' 'true' always;
            add_header 'Access-Control-Allow-Methods' 'GET, POST, OPTIONS';
            add_header 'Access-Control-Allow-Headers' 'Authorization,Accept,Origin,DNT,User-Agent,X-Requested-With,If-Modified-Since,Cache-Control,Content-Type,Range';
            add_header 'Access-Control-Expose-Headers' 'Content-Length,Content-Range';
        }
        if ($request_method = 'GET') {
            add_header 'Access-Control-Allow-Credentials' 'true' always;
            add_header 'Access-Control-Allow-Methods' 'GET, POST, OPTIONS';
            add_header 'Access-Control-Allow-Headers' 'Authorization,Accept,Origin,DNT,User-Agent,X-Requested-With,If-Modified-Since,Cache-Control,Content-Type,Range';
            add_header 'Access-Control-Expose-Headers' 'Content-Length,Content-Range';
        }
        proxy_redirect off;
        proxy_set_header host $host;
        proxy_set_header X-real-ip $remote_addr;
        proxy_set_header X-forward-for $proxy_add_x_forwarded_for;
        proxy_pass http://influxdb:9999;
      }
    }
}


```

### nginx Docker launch command line

If you have a _nginx_ container in the _Kitematic_, delete it. The above bindings are better done from the command line (bash, PowerShell, accessible from Kitematic's `Docker CLI` button) - it one single, long command line without line breaks:

`docker run --name nginx --restart=unless-stopped -p 8088:80 -p 8089:8089 --mount type=bind,source=C:\'Program Files (x86)'\OpenCPN\plugins\dashboard_tactics_pi\data\instrujs,target=/data/www --mount type=bind,source=~\nginx\nginx.conf,target=/etc/nginx/nginx.conf -d nginx`

### Link nginx container's network with InfluxDB's network

This avoids having complicated DNS or other arrangements since the instance of _nginx_ will see the instance of _InfluxDB v2_ as `influxdb` - this name is used in the proxy settings and if your _nginx_ is already running it is complaining about this.

This is best done using _Kitematic_ - it is this easy: I _nginx_ container's `Network` tab, in `Links` part, select _InfluxDB v2_ container under `NAME`. In `ALIAS`, type _influxdb_:

![2020-03-29_Kitematic_link_to_influxdb_network.png](2020-03-29_Kitematic_link_to_influxdb_network.png)

You may want to restart the _nginx_ container (you can use _Kitematic_'s button for that) to make sure that the error message relaed to missing _influxdb_ name has disappeared.

## InfluxDB v2

When this is written _InfluxDB v2_ is still in beta, but advanced one.

## Grafana

With _Grafana_ we want to use a separated volume to share it settings and other parameters in persistent manner, provided that we need to delete and restart the container.

### Link Granana container's network with InfluxDB's network

This avoids having complicated DNS or other arrangements and even the usage of _nginx_ proxy since the instance of _Grafana_ will see the instance of _InfluxDB v2_ as `influxdb` - this name will be used in its datasource settings, saved in the persistent volume. Therefore it is not good idea to change the following names when the container is recreated for some reason.

The link is best done using _Kitematic_ - it is this easy: I _nginx_ container's `Network` tab, in `Links` part, select _InfluxDB v2_ container under `NAME`. In `ALIAS`, type _influxdb_:

![2020-03-29_Kitematic_link_to_influxdb_network.png](2020-03-29_Kitematic_link_to_influxdb_network.png)

You may want to restart the _Grafana_ container (you can use _Kitematic_'s button for that) before making any configuration effort.